<a href="https://colab.research.google.com/github/natreis02/NLP-Text-Classification/blob/main/Transfer_Learning_and_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

BERT (Bidirectional Encoder Representations from Transformers) é um modelo de linguagem desenvolvido pela **Google** em 2018 que revolucionou o campo de Processamento de Linguagem Natural (NLP).


*   O BERT analisa o contexto de uma palavra considerando todas as palavras ao seu redor simultaneamente.
*   BERT é baseado na arquitetura Transformer, que utiliza mecanismos de atenção para pesar a importância de diferentes palavras em uma sequência.
*   O BERT é pré-treinado em grandes corpora de texto, como a Wikipedia e o LivroCorpus:


1.   Masked Language Model (MLM): Algumas palavras são mascaradas (substituídas por um token especial) e o modelo é treinado para prever essas palavras com base no contexto.

2.   Next Sentence Prediction (NSP): O modelo aprende a prever se duas sentenças estão relacionadas ou não, ajudando na compreensão de relações entre frases.

* Após o pré-treinamento, o modelo pode ser ajustado (fine-tuned) em tarefas específicas, como **classificação de texto**, **resposta a perguntas**, **Identificação de nomes**, **geração de texto**.








In [2]:
# Instalação das bibliotecas necessárias
!pip install transformers datasets torch

In [1]:
pip install datasets

In [4]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam

In [5]:
from datasets import load_dataset

In [6]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [7]:
dataset = load_dataset("ag_news")  # Manchete de notícias (esportiva, mundo, negócios e ciência)

In [8]:
# Verificar o número de exemplos no conjunto de dados original
print("Número total de exemplos no conjunto de dados de treinamento:", len(dataset['train']))
print("Número total de exemplos no conjunto de dados de teste:", len(dataset['test']))

Número total de exemplos no conjunto de dados de treinamento: 120000
Número total de exemplos no conjunto de dados de teste: 7600


In [9]:
from transformers import BertTokenizer, BertForSequenceClassification
# Carregar o tokenizador BERT
tokenizer = BertTokenizer.from_pretrained("distilbert-base-uncased") # Versão específica do modelo BERT. Transforma palavras ou frases em tokens (números).

# Tokenização e formatação dos dados
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True) #  aplica a função de tokenização a todos os dados no conjunto de dados (dataset).

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.


In [10]:
# Definindo o modelo BERT para classificação de sequência
model = BertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=4) # Cada sequência vai ser classicada e o modelo tem que ser definido.

You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'embeddings.LayerNorm.bias', 'embeddings.LayerNorm.weight', 'embeddings.position_embeddings.weight', 'embeddings.token_type_embeddings.weight', 'embeddings.word_embeddings.weight', 'encoder.layer.0.attention.output.LayerNorm.bias', 'encoder.layer.0.attention.output.LayerNorm.weight', 'encoder.layer.0.attention.output.dense.bias', 'encoder.layer.0.attention.output.dense.weight', 'encoder.layer.0.attention.self.key.bias', 'encoder.layer.0.attention.self.key.weight', 'encoder.layer.0.attention.self.query.bias', 'encoder.layer.0.attention.self.query.weight', 'encoder.layer.0.attention.self.value.bias', 'encoder.layer.0.attention.self.value.weight', 'enc

In [11]:
from transformers import TrainingArguments,Trainer

# Configurando os argumentos de treinamento
training_args = TrainingArguments(
    output_dir="./results",          # Diretório para salvar os resultados
    evaluation_strategy="epoch",     # Avaliar a cada época
    learning_rate=2e-5,              # Taxa de aprendizado
    per_device_train_batch_size=2,  # Tamanho do batch de treinamento
    per_device_eval_batch_size=2,   # Tamanho do batch de avaliação
    num_train_epochs=1,              # Número de épocas de treinamento
    weight_decay=0.01,               # Decaimento de peso para regularização
    no_cuda=True  # Adiciona isso para desativar o uso de GPUs
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [12]:
# 90% dos dados serão usados para treino e 10% para avaliação
tokenized_datasets = tokenized_datasets["train"].train_test_split(test_size=0.1)

In [13]:
# Reduzir o conjunto de dados de treinamento e teste
train_dataset = tokenized_datasets["train"].select(range(100))
eval_dataset = tokenized_datasets["test"].select(range(10))

# Verificar o número de exemplos após a redução
print("Número de exemplos no conjunto de dados de treinamento reduzido:", len(train_dataset))
print("Número de exemplos no conjunto de dados de teste reduzido:", len(eval_dataset))

Número de exemplos no conjunto de dados de treinamento reduzido: 100
Número de exemplos no conjunto de dados de teste reduzido: 10


In [14]:
# Instalar a biblioteca evaluate
!pip install evaluate

# Importar a nova biblioteca de métricas
import evaluate
accuracy_metric = evaluate.load("accuracy")

# Função para calcular as métricas
def compute_metrics(p):
    predictions, labels = p
    predictions = predictions.argmax(axis=-1)  # Seleciona a classe com maior probabilidade
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    return accuracy

In [15]:
# Definindo o trainer
trainer = Trainer(
    model=model,                         # O modelo BERT pré-treinado para classificação
    args=training_args,                  # Argumentos de treinamento
    train_dataset=train_dataset,         # Dados de treino
    eval_dataset=eval_dataset,           # Dados de avaliação
    tokenizer=tokenizer,                 # Tokenizador BERT
    compute_metrics=compute_metrics      # Função de métricas de avaliação
)

In [16]:
# Treinamento do modelo
trainer.train()

# Avaliação do modelo
trainer.evaluate()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.350877,0.300000


{'eval_loss': 1.3508766889572144,
 'eval_accuracy': 0.3,
 'eval_runtime': 17.4457,
 'eval_samples_per_second': 0.573,
 'eval_steps_per_second': 0.287,
 'epoch': 1.0}

In [29]:
import torch

# Exemplo de uma manchete externa
external_headline = "The local team wins the championship."

# Tokenizar a manchete
inputs = tokenizer(external_headline, return_tensors="pt", padding=True, truncation=True)

# Fazer a previsão
with torch.no_grad():  # Desativar o cálculo de gradientes
    outputs = model(**inputs)
    logits = outputs.logits

# Obter a classe prevista
predicted_class = logits.argmax(dim=-1).item()  # Obter a classe com a maior probabilidade

# Mapear a classe prevista para o nome da categoria
class_names = ["World", "Sports", "Business", "Science"]  # Ajuste conforme as classes do seu modelo
predicted_label = class_names[predicted_class]

# Exibir o resultado
print(f"A manchete: '{external_headline}' foi classificada como: {predicted_label}")

A manchete: 'The local team wins the championship.' foi classificada como: World
